In [3]:
import cv2
import mediapipe as mp
import numpy as np
import math

# 定義calArea函數，用來計算左眼、右眼與嘴巴中心三點之間的面積(將物件放大縮小用)
def calcArea(p1, p2, p3):
    
    # 取出各點的xy座標
    (x1, y1), (x2, y2), (x3, y3) = p1, p2, p3
    
    # 計算並回傳面積
    area = 0.5 * abs(x2 * y3 + x1 * y2 + x3 * y1 - x3 * y2 - x2 * y1 - x1 * y3)
    return area

# 定義calcAngle函數，用左眼、右眼來計算臉的角度(將物件旋轉用)
def calcAngle(x1, y1, x2, y2):
    
    # 計算並回傳角度
    x = x1 - x2
    y = y1 - y2
    z = math.sqrt(x*x + y*y)
    angle = round(math.asin(y/z)/math.pi * 180)
    
    return angle

# 定義rotateImage函數，傳入圖片、角度與旋轉中心，用來旋轉圖片
def rotateImage(image, angle, center):
    
    # 取得圖片的旋轉矩陣
    rot_mat = cv2.getRotationMatrix2D(center, angle, 1.0)
    
    # 對圖片進行變換
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    
    return result

# 定義alphaBlending函數，傳入前景、背景與alpha
def alphaBlending(foreground, background, alpha):

    # 將alpha遮罩標準化，讓數值介於0與1之間
    alpha = alpha.astype(float)/255
    
    # 將前景與遮罩相乘
    foreground1 = cv2.multiply(alpha.astype(float), foreground.astype(float))
    
    # 將背景與(1 - alpha)相乘
    background = cv2.multiply((1.0 - alpha).astype(float), background.astype(float))
    
    # 將兩張圖片相加
    result = cv2.add(foreground1, background)
    
    return result

# 定義putThings函數，傳入要放置的物件、背景、座標、角度與三點間的面積
def putThings(thing, background, coord, angle, area):
    
    # 設置hat的參數
    if thing == 'hat':
        
        # 設置物件要放置的位置
        (t_px, t_py) = ((coord['MOUTH_CENTER']['x'] - coord['NOSE_TIP']['x'])*1//3 + coord['NOSE_TIP']['x'] , 
                        coord['RIGHT_EYE']['y'] - (coord['NOSE_TIP']['y'])//7*4)
        
        # 讀入要放置的物件圖片
        foreground = cv2.imread("hat.png", cv2.IMREAD_UNCHANGED)
        
        # 計算圖片縮放大小
        scale = area / 13500
        
    # 設置mustache的參數
    elif thing == 'mustache':
        
        # 設置物件要放置的位置
        (t_px, t_py) = ((coord['MOUTH_CENTER']['x'] - coord['NOSE_TIP']['x'])*1//3 + coord['NOSE_TIP']['x'] , 
                    (coord['MOUTH_CENTER']['y'] - coord['NOSE_TIP']['y'])*1//3 + coord['NOSE_TIP']['y'] )
        
        # 讀入要放置的物件圖片
        foreground = cv2.imread("mustache.png", cv2.IMREAD_UNCHANGED)
        
        # 計算圖片縮放大小
        scale = area / 18000
    
    # 例外狀況，回傳背景
    else:
        return background
    
    # 將前景圖片縮放
    foreground = cv2.resize(foreground, None, fx = scale, fy = scale, interpolation= cv2.INTER_LINEAR)

    # 讀入背景長寬與通道數
    b_width, b_hight, b_ch = background.shape
    
    # 製作前景的地板圖片(與背景大小一致)
    foreground_ground = np.zeros((b_width, b_hight, 4), np.uint8)
    
    # 取得前景長寬與通道數
    f_width, f_hight,  f_ch = foreground.shape
    
    # 設定前景圖片要放置至前景的地板圖片的像素位置
    (thing_start_point_x, thing_end_point_x) = ((t_py - f_width//5*2), (t_py - f_width//5*2 + f_width))
    (thing_start_point_y, thing_end_point_y) = ((t_px - f_hight//2), (t_px - f_hight//2 + f_hight))
    
    # 處理例外狀況(要放置物件的位置超出畫面範圍之類的)
    if  thing_start_point_x < 0 or  thing_end_point_x > b_width or thing_start_point_y < 0 or  thing_end_point_y > b_hight:
        
        # 要放置的物件受到上方畫框裁切
        if thing_start_point_x < 0 and thing_end_point_x > 0 and thing_start_point_y > 0 and thing_end_point_y < b_hight:
            
            # 裁切前景圖片
            foreground = foreground[( 0 - thing_start_point_x ):,:,:]
            # 將裁切後的前景圖片放至前景的地板圖片中
            foreground_ground[ 0 : f_width + thing_start_point_x, thing_start_point_y : thing_end_point_y] = foreground
        
        # 要放置的物件受到下方畫框裁切
        elif thing_start_point_x < b_width and thing_end_point_x > b_width and thing_start_point_y > 0 and thing_end_point_y < b_hight:
            
            foreground = foreground[ : (b_width - thing_start_point_x),:,:]
            
            foreground_ground[ thing_start_point_x : , thing_start_point_y : thing_end_point_y] = foreground
        
        # 要放置的物件受到右方畫框裁切
        elif thing_start_point_y < 0 and thing_end_point_y > 0 and thing_start_point_x > 0 and thing_end_point_x < b_width:
            
            foreground = foreground[:,( 0 - thing_start_point_y ):,:]
            
            foreground_ground[ thing_start_point_x : thing_end_point_x , 0 : f_hight + thing_start_point_y] = foreground
        
        # 要放置的物件受到左方畫框裁切
        elif thing_start_point_y < b_hight and thing_end_point_y > b_hight and thing_start_point_x > 0 and thing_end_point_x < b_width:
            
            foreground = foreground[:,:(b_hight - thing_start_point_y),:]

            foreground_ground[ thing_start_point_x : thing_end_point_x , thing_start_point_y : ] = foreground
        
        # 例外狀況，回傳背景圖
        else:
            return background
        
    else:
        # 直接將前景圖片放至前景的地板圖片中
        foreground_ground[ thing_start_point_x : thing_end_point_x, thing_start_point_y : thing_end_point_y] = foreground
    
    # 設置旋轉中心點，我這次以鬍子的位置為旋轉中心
    center = ((coord['MOUTH_CENTER']['x'] - coord['NOSE_TIP']['x'])*1//3 + coord['NOSE_TIP']['x'] , 
                (coord['MOUTH_CENTER']['y'] - coord['NOSE_TIP']['y'])*1//3 + coord['NOSE_TIP']['y'] )
    
    # 旋轉前景圖片
    foreground_ground = rotateImage(foreground_ground, angle, center)
    
    # 製作 3 channel 的 alpha遮罩
    alpha = cv2.merge([foreground_ground[:,:,3], foreground_ground[:,:,3], foreground_ground[:,:,3]])
    
    # 將前景圖片轉為 3 channel (為了混合圖片)
    foreground_ground = foreground_ground[:,:,:3]
    
    # 將前景與背景做alpha blending
    output = alphaBlending(foreground_ground, background, alpha)
    
    return output.astype(np.uint8)
    
# 打開攝影機
cap =  cv2.VideoCapture(1)

# 設定做臉部偵測
mp_face_detection = mp.solutions.face_detection

# 開始臉部偵測
with mp_face_detection.FaceDetection(min_detection_confidence=0.5) as face_detection:
  
  while True:

    # 讀入影格
    ret, frame = cap.read()
    
    # 檢查是否正確讀入
    if ret == False:
        print("Cannot read frame")
        break
    
    # 取得畫面長寬
    height, width = frame.shape[:2]

    # 將圖片由 BGR 轉為 RGB
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 做臉部偵測
    results = face_detection.process(frame)

    if not results.detections:
        continue
    
    # 複製影格
    result_frame = frame.copy()
    
    # 將圖片由 RGB 轉為 BGR
    result_frame = cv2.cvtColor(result_frame, cv2.COLOR_RGB2BGR)
    
    # 走訪每一張臉
    for detection in results.detections:
        
        coord = {}
        
        # 走訪臉中每一個點的位置
        for kp in mp.solutions.face_detection.FaceKeyPoint:

            # 走訪每一個 FaceKeyPoint
            keypoint = mp.solutions.face_detection.get_key_point(detection, kp)

            # 將 keypoint 中的長寬乘上影像長度、寬度，得到在影像中實際對應的位置
            x = int(keypoint.x * width)
            y = int(keypoint.y * height)
            
            # 存入座標清單
            coord[kp.name] = {"x" : x , "y" : y}
        
        # 計算臉的角度(使用兩隻眼睛的座標計算)
        angle = calcAngle(coord['RIGHT_EYE']['x'], coord['RIGHT_EYE']['y'], 
                           coord['LEFT_EYE']['x'], coord['LEFT_EYE']['y'])
        
        # 計算右眼、左眼與嘴巴中心點，三點間的面積
        area = calcArea((coord['RIGHT_EYE']['x'], coord['RIGHT_EYE']['y']), 
                        (coord['LEFT_EYE']['x'], coord['LEFT_EYE']['y']),
                        (coord['MOUTH_CENTER']['x'], coord['MOUTH_CENTER']['y']))
        
        # 將 mustache 放至圖片上
        result_frame = putThings('mustache', result_frame, coord, angle, area)
        
        # 將 hat 放至圖片上
        result_frame = putThings('hat', result_frame, coord, angle, area)

        # 標出右眼、左眼與嘴巴中心點的位置
        cv2.circle(result_frame, (coord['RIGHT_EYE']['x'],coord['RIGHT_EYE']['y']), 5, (0,255,0), -1)
        cv2.circle(result_frame, (coord['LEFT_EYE']['x'],coord['LEFT_EYE']['y']), 5, (0,255,0), -1)
        cv2.circle(result_frame, (coord['MOUTH_CENTER']['x'],coord['MOUTH_CENTER']['y']), 5, (0,255,0), -1)
        
        # 顯示結果
        cv2.imshow('results', result_frame)

    c = cv2.waitKey(1)
    
    if c == 27:
        cv2.destroyAllWindows()
        break

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
